#  1. zápočtová úloha z 01RAD 2022/23

## Popis úlohy

V tomto úkolu je cílem provést předzpracování datového souboru, jeho vizualizaci a jednoduchou lineární regresní úlohu, kde se budeme zajímat o ceny nemovitostí. Za tímto účelem využijeme datový set *saratosa_hoouses* z knihovny *moderndive* obsahující výběr 1057 domů.



## Podmínky a body

Úkol i protokol vypracujte samostatně. Pokud na řešení nějaké úlohy budete přesto s někým spolupracovat, radit se, nezapomeňte to u odpovědi na danou otázku uvést. Tato zápočtová úloha obsahuje 10 otázek po 1 bodu. Celkem za 3 zápočtové úlohy bude možné získat 30 bodů, přičemž pro získání zápočtu je potřeba více jak 20 bodů. Další dodatečné body mohu případně individuálně udělit za řešení mini domácích úkolů z jednotlivých hodin.



## Odevzdání

Protokol ve formátu Rmd+pdf, nebo jako Jupyter notebook (idealně odkaz na gitlab s možností spustit v Colabu) nejpozději do 1. 11. 2022.



### Předzpracování dat:




In [ ]:
list_of_packages <- c("tidyverse", "MASS","GGally","moderndive")
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
missing_packages
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)

In [ ]:
? saratoga_houses

**Data**

In [ ]:
head(saratoga_houses)

## Otázka 01

Zjistěte, zdali data neobsahují chybějící hodnoty *NA*. Pokud ano, tak rozhodněte zdali můžete příslušná pozorování z dat odstranit a proč. Které proměnné jsou kvantitativní a které kvalitativní? Jeli možno některé zařadit do obou skupin, pro kterou byste se rozhodli? Které proměnné je možné pužít jako faktorové ordinální a jaké jako faktorové nominální a proč? Spočtěte základní statistiky pro jednotlivé proměnné.

### Řešení 01:

Pomocí funkce summary vidíme, že proměnná *lot_size* obsahuje 9 hodnot *NA*. Jelikož jde pouze o 9 hodnot z celkových 1057, myslím, že tato pozorování můžeme z dat odstranit.

In [ ]:
summary(saratoga_houses)

na_hodnoty <- saratoga_houses[rowSums(is.na(saratoga_houses))>0,]
na_hodnoty

saratoga_houses <- saratoga_houses[complete.cases(saratoga_houses),]

Kvantitativní proměnné jsou *price, living_area, bathrooms, bedrooms, fireplaces, lot_size* a *age*. Kvalitativní proměnnou je *fireplace*. Do kvalitativních by šly také zařadit proměnné *bathrooms, bedrooms* a *fireplaces*, ale ponechala bych je jako kvantitativní.

Jako faktorové ordinální lze použít proměnné *bathrooms, bedrooms* a *fireplaces* (počet pokojů/koupelen/krbů lze seřadit podle velikosti a porovnávat mezi sebou). Do faktorových nominálních lze zařadit proměnnou *fireplace*.

Základní statistiky zjistíme pomocí funkce *summary*, nebo lze střední hodnoty spočítat pomocí funkce *colMeans*. Dále je ještě třeba spočítat rozptyly a směrodatné odchylky.

In [ ]:
str(saratoga_houses)
summary(saratoga_houses)

colMeans(saratoga_houses[sapply(saratoga_houses,is.numeric)])

c(sd(saratoga_houses$price), var(saratoga_houses$price))
c(sd(saratoga_houses$living_area),var(saratoga_houses$living_area))
c(sd(saratoga_houses$bathrooms), var(saratoga_houses$bathrooms))
c(sd(saratoga_houses$bedrooms), var(saratoga_houses$bedrooms))
c(sd(saratoga_houses$fireplaces), var(saratoga_houses$fireplaces))
c(sd(saratoga_houses$lot_size),var(saratoga_houses$lot_size))
c(sd(saratoga_houses$age),var(saratoga_houses$age))

## Otázka 02

Chceme koupit nemovitost v zahraničí a průzkumem trhu jsme obdřeli předchozí data set *saratoga_houses*. Jelikož ale máme přesnější požadavky a nerozumíme imperiálním jednotkám, potřebujeme data upravit:

* Převeďte cenu nemovitostí z dolarů na koruny v tisících a plochu pozemku a obytnou plochu z akrů a čtverečích stop na $m^2$. (chceck describtion by *? saratoga_houses*) 
* Vyberte jen nemovitosti starší 10 let a mladší 50 let, jejichž cena je menší než 7500000 Kč, a plocha pozemku je mezi 500 a 5000 $m^2$.
* Počet koupelen a počet pokojů převeďte na faktorové proměnné o 3 úrovních.

**Dále pracujte jen s takto omezeným datasetem a s proměnnýma *cena*, *plocha_obytna*, *plocha_pozemku* *pocet_pokoju*,*stari_domu*, *pocet_koupelen*, *krb*.**

### Řešení 02:

In [ ]:
saratoga_houses$price <- saratoga_houses$price*(24.8/1000)
saratoga_houses$living_area <- saratoga_houses$living_area*0.092903
saratoga_houses$lot_size <- saratoga_houses$lot_size*4046.86


In [ ]:
houses_vybrane <- saratoga_houses[saratoga_houses$age > 10 & saratoga_houses$age < 50 & saratoga_houses$price < 7500 & saratoga_houses$lot_size > 500 & saratoga_houses$lot_size < 5000, ]
head(houses_vybrane)

Vytvoříme si nové proměnné *bedrooms2* a *bathrooms2*, které převedeme pomocí funkce *cut* na faktorové proměnné o 3 úrovních.

In [ ]:
houses_vybrane$bedrooms2 <- cut(houses_vybrane$bedrooms, c(0,2.5,3.5,5) , labels = c("1-2","3","4-5"))
houses_vybrane$bathrooms2 <- cut(houses_vybrane$bathrooms, c(0,1.1, 2.1, 5), labels = c("1","1.5-2","2.5-4.5"))

head(houses_vybrane)


## Otázka 03 

* Porovnejte průměry cen nemovitostí s krbem a bez krbu a otestujte, zdali na hladině významnosti $\alpha = 0.01$ je průměrná cena nemovitostí s krbem větší než průměrná cena nemovitostí bez krbu.

### Řešení 03:


Můžeme nahlédnout do boxplotu na hodnoty cen nemovitostí s krbem a bez krbu.

In [ ]:
domy_bez <- houses_vybrane[houses_vybrane$fireplace == FALSE,]
domy_s <- houses_vybrane[houses_vybrane$fireplace == TRUE, ]

mean(domy_bez$price)
mean(domy_s$price)

boxplot(domy_s$price, domy_bez$price, ylab = "cena", main = "Porovnání cen nemovitostí s krbem a bez", names = c("domy s krbem", "domy bez krbu"))

Otestujme nejprve normalitu dat, pomocí Shapiro-Wilkova testu normality. Vychází nám p-hodnota menší než naše α, a tedy zamítáme normalitu dat. 





In [ ]:
shapiro.test(domy_bez$price)
shapiro.test(domy_s$price)


Chceme testovat hypotézu $H_0: μ_s \geq μ_{bez}$ oproti alternativě $H_1: μ_s < μ_{bez}$. p-hodnota vyšla 1, což je větší než naše $α = 0.01$, a tedy hypotézu, že cena nemovitosti s krbem je vyšší než cena nemovitosti bez krbu, nezamítáme.

In [ ]:
wilcox.test(data=houses_vybrane, price ~ fireplace, alternative = c("greater"))

#wilcox.test(domy_s$price, domy_bez$price, alternative = c("less"))

# Vizualizace dat

## Otázka 04 

* Vykreslete scatterploty pro všechny numerické proměnné, kde bude barevně rozlišeno, zdali se jedná o nemovitost s krbem, nebo bez krbu.
*  Pro proměnné *pocet_pokoju* a *pocet_koupelen* a *krb* vykreslete krabicové diagramy (nebo violin ploty), kde odezvou bude *cena*.
* Pro proměnnou *cena* vykreslete histogram spolu s jádrovým odhadem hustoty.


Vykreslila jsem pouze scatterploty pro závislost ceny nemovitosti na *living_area, lot_size* a *age*, jelikož mi to dává největší smysl. Samozřejmě by šlo stejným způsobem vykreslit všechny možné kombinace závislostí numerických proměnných.

In [ ]:
ggplot(houses_vybrane,aes(x = living_area, y = price, color = fireplace)) + 
  geom_point()

ggplot(houses_vybrane,aes(x= lot_size, y = price, color = fireplace)) + 
  geom_point()

ggplot(houses_vybrane,aes(x=age,y=price, color = fireplace)) + 
  geom_point()



Krabicové diagramy jsem vykreslila pro dříve vytvořené faktorové proměnné o 3 úrovních (*bedrooms2*, *bathrooms2*), tak i (po převedení na faktorové proměnné) pro zadané proměnné *bedrooms* a *bathrooms*.

In [ ]:
houses_vybrane$bedrooms <- as.factor(houses_vybrane$bedrooms)
houses_vybrane$bathrooms <- as.factor(houses_vybrane$bathrooms)

In [ ]:
ggplot(houses_vybrane, aes(x = bedrooms, y= price))+ geom_boxplot()
ggplot(houses_vybrane, aes (x=bathrooms, y=price))+geom_boxplot()

In [ ]:
ggplot(houses_vybrane, aes(x=bedrooms2, y= price)) + geom_boxplot()
ggplot(houses_vybrane, aes(x=bathrooms2, y= price)) + geom_boxplot()
ggplot(houses_vybrane, aes(x=fireplace,y=price))+geom_boxplot()

In [ ]:
ggplot(houses_vybrane, aes(x = price))+
  geom_histogram(aes(y = ..density..),color = "black",fill = "white", bins = 15) +
  geom_density(kernel = "gaussian", alpha = 0.2, color = "blue")

#nebo

hist(houses_vybrane$price, freq = FALSE, xlab = "cena", main="histogram ceny")
lines(density(houses_vybrane$price), col = "blue", lwd = 2)



## Otázka 05

Pro kombinace faktorizovaných proměnných *pocet_pokoju*, *pocet_koupelen*  vykreslete cenu nemovitosti, aby bylo na obrázku vidět, jestli se v průměru liší ceny nemovitostí majících více pokojů, nebo více koupelen a zdali jsou zastoupeny všechny kombiance všech úrovních pro dvě zmíněné faktorové proměnné.

In [ ]:
ggplot(houses_vybrane, aes(x=bedrooms2, y=price, fill=bathrooms2)) + 
        geom_boxplot(size = 0.7, notch = F) +
        xlab("Bedrooms") +
        ylab("Price") +
        theme_bw() +
        geom_jitter(aes(bedrooms2,price,color=bathrooms2),
                    position=position_jitter(width=0.3, height=0),
                    alpha = 0.6,
                    size = 1,
                    show.legend=F) +
        stat_summary(fun=mean, geom="point", shape = 23, size=3)



Pro takto definované faktorové proměnné o 3 úrovních jsou zastoupené všechny kombinace.

## Otázka 06

Pro nemovitosti s dvěma ložnicema vykreslete závislost ceny na obytné ploše nemovitosti, kde jednotlivé události označíte barvou podle toho zdali mají krb a velikost bodů v grafu bude odpovídat počtu koupelen (pro tuto úlohu je lepší vzít počet koupelen jako numerickou proměnnou).

**Dále pracujte jen s nemovitostmi se dvěma ložnicemi.**

In [ ]:
houses_2bedrooms <- houses_vybrane[houses_vybrane$bedrooms == 2,]
head(houses_2bedrooms)

houses_2bedrooms$bathrooms<- as.numeric(houses_2bedrooms$bathrooms)

ggplot(houses_2bedrooms, aes(x=living_area, y=price, color = fireplace, size = bathrooms)) + geom_point()

houses_2bedrooms$bathrooms <- as.factor(houses_2bedrooms$bathrooms)


# Jednoduchý lineární model

## Otázka 07

Sestavte jednoduchý regresní model (s i bez interceptu), kde vysvětlovaná proměnná
bude cena nemovitosti a vysvětlující obytná plocha.   Spočtěte pro oba modely $R^2$ a $F$ statistiky, co nám o modelech říkají. Vyberte jeden z nich a zdůvodněte proč ho preferujete.

Na základě zvoleného modelu odpovězte, zdali cena nemovitosti závisí na obytné ploše  a pokud ano, o kolik se změní očekávaná cena pro nemovitost s obytnou plochou zvětšenou o $20 m^2$? 

In [ ]:
model_without_intercept <- lm(price ~ -1 + living_area, data = houses_2bedrooms)
summary(model_without_intercept)

model_with_intercept <- lm(price ~ living_area, data = houses_2bedrooms)
summary(model_with_intercept)


ggplot(houses_2bedrooms, aes(x=living_area, y = price))+
  geom_point()+
  geom_abline(aes(intercept = model_with_intercept$coefficients[1], slope = model_with_intercept$coefficients[2], color = "s interceptem"))+
  geom_abline(aes(intercept = 0, slope = model_without_intercept$coefficients[1], color = "bez interceptu"))

$R^2$ statistika nám říká, jak dobře data odpovídají regresnímu modelu. Dále platí, že čím je F statistika vyšší, tím je model lepší.


*   V modelu bez interceptu je $R^2$: 0.9427, $F$: 1432 a p-hodnota: $2.2*10^{-16}$.
*   V modelu s interceptem vychází $R^2$: 0.4549, $F$: 71.78 a p-hodnota: $5.84*10^{-13}$.

V modelu nám nevystupuje proměnná celková rozloha, na které by cena mohla také záviset. Model tedy nemusí pro nulovou obytnou plochu dávat nulovou cenu, jelikož můžeme mít pouze pozemek, za který je také třeba zaplatit. A tedy model závislosti ceny na obytné ploše nemusí procházet počátkem.

V modelu bez interceptu sice data odpovídají lépe modelu, ale model nemusí například tak dobře predikovat hodnoty pro nová data.

Vybrala bych proto model s interceptem.






In [ ]:
delta_living_area <- 20
delta_price <- (model_with_intercept$coefficients[2]*delta_living_area)
delta_price

Dostaneme model: $Y = 563.75 + 19.117*X$ $[$jednotka: tisíc Kč$]$, kde proměnná $X$...living_area je statisticky významná (p-hodnota: $5.84*10^{-13}$), a tedy cena nemovitosti na obytné ploše závisí.

Zvýší-li se obytná plocha o 20 $m^2$, tak se očekávaná cena zvýší o 382.34 tisíc Kč.

## Otázka 08
Sestavte jednoduchý linární model jako v předchozí otázce pro nemovitosti s krbem a bez krbu. Jaký model vykazuje silnější linearní vztah mezi cenou a obytnou plochou? O kolik cena s rostoucí obytnou plochou pro nemovitosti s krbem roste rychleji než pro nemovitosti bez krbu?

Spočtěte 95% konfidenční intervaly pro regresní koeficienty popisující sklon regresní přímky v obou modelech a zjistěte, zdali se protínají. Co z toho můžeme vyvozovat?

Na základě těchto modelů zjistěte o kolik procent bude mít průměrná nemovitost s krbem a obytnou plochou $160m^2$ vyšší očekávanou cenu než průměrná nemovitost o stejné obytné ploše, ale bez krbu.

Budu tedy uvažovat už jen model s interceptem, pro který jsem se v předchozí úloze rozhodla. Sestavíme vykreslíme tedy model s interceptem pro nemovitosti s krbem a bez krbu.

In [ ]:
model_without_fireplace <- lm(price ~ living_area, data = houses_2bedrooms[houses_2bedrooms$fireplace == FALSE, ])
summary(model_without_fireplace)

model_with_fireplace <- lm(price ~ living_area, data = houses_2bedrooms[houses_2bedrooms$fireplace == TRUE, ])
summary(model_with_fireplace)

ggplot(houses_2bedrooms, aes(x=living_area, y = price))+
  geom_point()+
  geom_abline(aes(intercept = model_without_fireplace$coefficients[1], slope = model_without_fireplace$coefficients[2], color = "bez krbu"))+
  geom_abline(aes(intercept = model_with_fireplace$coefficients[1], slope = model_with_fireplace$coefficients[2], color = "s krbem"))

model_with_fireplace$coefficients[2]-model_without_fireplace$coefficients[2]  


Velká hodnota $R^2$ indikuje přibližně lineární vztah mezi proměnnými. Silnější lineární vztah mezi cenou a obytnou plochou tak vykazuje model s krbem, jelikož má vyšší $R^2$.

Cena nemovitostí skrbem roste rychleji o 11.88 tisíc Kč na $m^2$.

In [ ]:
confint(model_without_fireplace)
confint(model_with_fireplace)

ggplot(houses_2bedrooms, aes(x = living_area, y = price , color = fireplace))+
  geom_point()+
  geom_smooth(method = lm, formula = y ~ x, aes(color = fireplace))


95% konfidenční interval pro regresní koeficient popisující sklon regresní přímky pro model s krbem je [13.55598 ;	32.45133] a pro model bez krbu [6.8569 ;	15.38681]. 

Konfidenční intervaly se pro nějaké hodnoty protínají, a tedy rozdíl mezi skupinami zde není statisticky významný.

Chceme vyřešit, o kolik procent je cena nemovitosti s krbem vyšší než cena nemovitosti bez krbu, je-li obytná plocha 160 $m^2$. Počítáme tedy $[(Y_s - Y_{bez})/Y_{bez}] * 100$ a vychází, že cena nemovitosti s krbem je o 25.86% vyšší.

In [ ]:
X <- 160
Y_s <- model_with_fireplace$coefficients[1]+model_with_fireplace$coefficients[2]*X
Y_bez <- model_without_fireplace$coefficients[1]+model_without_fireplace$coefficients[2]*X

((Y_s-Y_bez)/Y_bez)*100


## Otázka 9

Vykreslete scatterplot obytné plochy a ceny nemovitostí. Do tohoto grafu vykreslete regresní přímky vybraných modelů pro nemovitosti s krbem a bez něho, jednoltivé body i regresní přímky označte barvou podle toho k jaké skupině přísluší.

Sestrojte 90% konfidenční intervaly okolo očekávaných cen pro jednoltivé skupiny a na jejich základě rozhodněte, zdali a jak se očekávané ceny budou lišit pro nemovitosti s obytnou plochou menší než $120m^2$. Je to porovnávání správné? Zdůvoněte.

In [ ]:
ggplot(data = houses_2bedrooms, aes(x=living_area, y = price, color = fireplace))+
  geom_point() +
  geom_smooth(method = lm, aes(color = fireplace), se =F)

In [ ]:
confint(model_without_fireplace, level = 0.9)
confint(model_with_fireplace, level = 0.9)


new_pred_s = predict(model_with_fireplace, newdata = data.frame(living_area = seq(0,500,1)), interval = "prediction", level = 0.9)
new_pred_bez = predict(model_without_fireplace, newdata = data.frame(living_area = seq(0,500,1)), interval = "prediction", level = 0.9)

par(mfrow = c(1,2))
plot(price ~ living_area, data = houses_2bedrooms[houses_2bedrooms$fireplace == TRUE,],main = "90% IP pro ceny nemovitostí s krbem" )
lines(seq(0, 500, 1), new_pred_s[,1], col='black')
lines(seq(0, 500, 1), new_pred_s[,2], col='red')
lines(seq(0, 500, 1), new_pred_s[,3], col='red')

plot(price ~ living_area, data = houses_2bedrooms[houses_2bedrooms$fireplace == FALSE,], main = "90% IP pro ceny nemovitostí bez krbu")
lines(seq(0, 500, 1), new_pred_bez[,1], col='black')
lines(seq(0, 500, 1), new_pred_bez[,2], col='red')
lines(seq(0, 500, 1), new_pred_bez[,3], col='red')

# nebo pomocí ggplot2

predictions_s <- as.data.frame(predict(model_with_fireplace, interval = "predict", level = 0.9))
predictions_bez <- as.data.frame(predict(model_without_fireplace, interval = "predict", level = 0.9))

ggplot(data = cbind(houses_2bedrooms[houses_2bedrooms$fireplace == TRUE,],predictions_s), aes(x=living_area, y=price)) +
    geom_point() +
    geom_smooth(method=lm , color="red", se=TRUE) +
    geom_line( aes(y = lwr), col = "blue", linetype = "dashed") + 
    geom_line( aes(y = upr), col = "blue", linetype = "dashed") + 
    theme_bw()

ggplot(data = cbind(houses_2bedrooms[houses_2bedrooms$fireplace == FALSE,],predictions_bez), aes(x=living_area, y=price)) +
    geom_point() +
    geom_smooth(method=lm , color="red", se=TRUE) +
    geom_line( aes(y = lwr), col = "blue", linetype = "dashed") + 
    geom_line( aes(y = upr), col = "blue", linetype = "dashed") + 
    theme_bw()




## Otázka 10

Vykreslete histogramy pro rezidua modelů z předchozí otázky. Proložte je hustotou normálního rozdělení s nulovou střední hodnotou a rozptylem odpovídajícím $\hat{\sigma}^2$ z jednotlivých modelů.

Co výsledný graf říka o n našich modelech a je toto ověření dostatečné pro validaci model?

Navrněte další úpravy modelu za cílem co nejlépe predikvoat cenu nemovitosti.

In [ ]:
par(mfrow=c(1,2))
hist(model_with_fireplace$residuals, main = "Histogram reziduí modelu s krbem", freq = F, xlab = "rezidua", )
x=seq(-3000,3000, length=10000)
y=dnorm(x, mean = 0, sd = sd(model_with_fireplace$residuals))
lines(x,y,col="blue", lwd=2, lty = 1)

hist(model_without_fireplace$residuals, main = "Histogram reziduí modelu bez krbu", freq = F, xlab = "rezidua", ylim = c(0,0.001))
x=seq(-3000,3000, length=10000)
y=dnorm(x, mean = 0, sd = sd(model_without_fireplace$residuals))
lines(x,y,col="blue", lwd=2, lty = 1)

qqnorm(model_with_fireplace$residuals)
qqline(model_with_fireplace$residuals)
qqnorm(model_without_fireplace$residuals)
qqline(model_without_fireplace$residuals)

shapiro.test(model_with_fireplace$residuals)
shapiro.test(model_without_fireplace$residuals)

  

Samotný pohled na histogramy reziduí nám může napovědět, že mají normální rozdělení. Lépe je normalita vidět pomocí QQ-plotu nebo můžeme použít Shapiro-Wilkův test normality. Ty nám ukazují normalitu reziduí. Normalita reziduí znamená, že náš model je platný.

Pro lepší predikci je možné do modelu přidat další proměnné.
